In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import argparse
from torch.utils.tensorboard import SummaryWriter

from dataset import load_data, GtzanDataset
from baseline_cnn import CNN

In [3]:
class Config():
    def __init__(self):
        self.batch_size = 32
        self.epochs = 5
        self.lr = 0.01
args = Config()

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
is_test = True

dev_reporter = Reporter('baseline')

# Load Data
GTZAN = load_data(is_test=is_test)

train_dataset = GtzanDataset(GTZAN.train_x, GTZAN.train_y, train=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=args.batch_size)

test_dataset = GtzanDataset(GTZAN.test_x, GTZAN.test_y, train=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=args.batch_size, shuffle=True)

# Initialize network
model = CNN('yeet', get_report_data=True).to(device)

dev_reporter.record_first_batch(model, train_loader)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=args.lr)
# Train Network
for epoch in range(args.epochs):
    print('epoch num ', epoch)

    # reset epoch reporter
    total_loss = 0
    total_correct = 0


    for batch_idx, (data, targets) in tqdm(enumerate(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        preds = model(data)
        loss = criterion(preds, targets)

        # add to total loss and accuracy to epoch report
        total_loss += loss.item()
        total_correct += get_num_correct(preds, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
    tb.add_scalar("Loss", total_loss, epoch)
    tb.add_scalar("Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct/ len(train_set), epoch)

    tb.add_histogram("conv1.bias", model.conv1.bias, epoch)
    tb.add_histogram("conv1.weight", model.conv1.weight, epoch)
    tb.add_histogram("conv2.bias", model.conv2.bias, epoch)
    tb.add_histogram("conv2.weight", model.conv2.weight, epoch)

keeping log
torch.Size([1, 128, 2330])


SyntaxError: 'break' outside loop (<ipython-input-46-444879f2e801>, line 19)

In [23]:

@torch.no_grad()
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    for batch in tqdm(loader):
        images, labels = batch

        preds = model(images)
        all_preds = torch.cat(
            (all_preds, preds)
            ,dim=0
        )
    return all_preds

def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(torch.from_numpy(labels)).sum().item()

In [47]:
from sklearn.metrics import confusion_matrix
from reporter_utils import plot_confusion_matrix
import torchvision

possible_names = ['baseline', 'segmented']

class Reporter():
    def __init__(self, name):
        if name not in possible_names:
            raise ValueError('name is not recognized from possible experiment names')
        self.confusion_matrix = None
        # self.train_summary_writer = SummaryWriter('logs/tensorboard/' + name + '/train')
        # self.test_summary_writer = SummaryWriter('logs/tensorboard/' + name + '/test')

    def set_post_training_values(self, model, train_set, test_set):
        self.model = model
        self.train_set = train_set
        self.test_set = test_set
    def report_on_model(self):
        train_loader = torch.utils.data.DataLoader(self.train_set, batch_size=64)
        test_loader = torch.utils.data.DataLoader(self.test_set, batch_size=64)
        
        print('\n\nGetting correct predictions...')
        self.train_predictions = get_all_preds(self.model, train_loader)
        self.test_predictions = get_all_preds(self.model, test_loader)

        train_num_correct = get_num_correct(self.train_predictions, self.train_set.targets)
        test_num_correct = get_num_correct(self.test_predictions, self.test_set.targets)
        print('train_num_correct', train_num_correct)
        print('test_num_correct', test_num_correct)
        self.confusion_matrix = confusion_matrix(
                                    self.train_set.targets,
                                    self.train_predictions.argmax(dim=1))
    
    def show_confusion_matrix(self):
        if self.confusion_matrix is not None:
            plot_confusion_matrix(self.confusion_matrix)
        else:
            raise ValueError(
                'confusion matrix is not generated yet, please run Reporter.report_on_model() to generate it.')
    def record_first_batch(self, model, loader):
        with torch.no_grad():
            images, labels = next(iter(loader))
            grid = torchvision.utils.make_grid(images.unsqueeze(1))
            first_spectrogram = images.unsqueeze(1)[0,:,:,:]
            print(first_spectrogram.shape)            
            # print(type(grid))
            # print(grid)
            tb = SummaryWriter()

            tb.add_image('images', first_spectrogram)
            tb.add_graph(model, grid)
            tb.close()

In [31]:
reporter = Reporter('baseline')

In [32]:
reporter.set_post_training_values(model, train_dataset, test_dataset)


In [34]:
reporter.report_on_model()

  0%|          | 0/4 [00:00<?, ?it/s]

Getting correct predictions...
100%|██████████| 1/1 [00:00<00:00,  6.22it/s]train_num_correct 33
test_num_correct 0



In [1]:
# type()
reporter.confusion_matrix.shape

NameError: name 'reporter' is not defined